In [1]:
import numpy as np
from scipy.optimize import basinhopping, differential_evolution, dual_annealing
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from PIL import Image

# Define the sequences
P = np.array([1.0, 2.0, 3.0, 4.0])
Q1 = np.array([1.5, 2.5, 2.8, 4.2])

# Small epsilon to avoid NaN calculations
epsilon = 1e-10

# Define various distance measures
def cosine_distance(Q, P=P):
    similarity = np.dot(P, Q) / (np.linalg.norm(P) * np.linalg.norm(Q))
    distance = 1 - similarity
    return distance

def itakura_saito_distance(Q, P=P):
    return np.sum((P / (Q + epsilon)) - np.log((P + epsilon) / (Q + epsilon)) - 1)

def spectral_convergence_distance(Q, P=P):
    return np.linalg.norm(P - Q) / (np.linalg.norm(P) + epsilon)

def euclidean_distance(Q, P=P):
    return np.linalg.norm(P - Q)

def manhattan_distance(Q, P=P):
    return np.sum(np.abs(P - Q))

def kullback_leibler_divergence(Q, P=P):
    return np.sum(P * np.log((P + epsilon) / (Q + epsilon)))

def pearson_correlation_distance(Q, P=P):
    return 1 - np.corrcoef(P, Q)[0, 1]

# Choose the distance function
distance_function = itakura_saito_distance  # Change this to use a different distance measure
distance_function_name = distance_function.__name__

# Control the maximum number of iterations via variable
max_iterations = 25

# Initialize lists to store results for each iteration
bh_results = [Q1.copy()]
de_results = [Q1.copy()]
da_results = [Q1.copy()]

# Callback function to store each iteration's results
def store_results(xk, f=None, accept=None):
    if len(bh_results) < max_iterations:
        bh_results.append(np.copy(xk))

# Run Basin-hopping optimization
minimizer_kwargs = {"method": "L-BFGS-B"}
result_bh = basinhopping(distance_function, Q1, minimizer_kwargs=minimizer_kwargs, niter=max_iterations, callback=store_results)

# Fill remaining iterations with the last result
while len(bh_results) < max_iterations:
    bh_results.append(bh_results[-1])

# Run Differential Evolution optimization and store each iteration's results
def de_callback(xk, convergence=None):
    if len(de_results) < max_iterations:
        de_results.append(np.copy(xk))

result_de = differential_evolution(distance_function, bounds=[(0, 5)] * len(P), maxiter=max_iterations, callback=de_callback)

# Fill remaining iterations with the last result
while len(de_results) < max_iterations:
    de_results.append(de_results[-1])

# Run Dual Annealing optimization and store each iteration's results
def da_callback(xk, f, context=None):
    if len(da_results) < max_iterations:
        da_results.append(np.copy(xk))

result_da = dual_annealing(distance_function, bounds=[(0, 5)] * len(P), maxiter=max_iterations, callback=da_callback)

# Fill remaining iterations with the last result
while len(da_results) < max_iterations:
    da_results.append(da_results[-1])

# Create animations
fig, ax = plt.subplots(figsize=(10, 6))

def animate(i, results, method_name, distance_function_name):
    ax.clear()
    ax.plot(P, 'bo-', label='Reference (P)')
    ax.plot(Q1, 'go-', label='Start (Q1)')
    ax.plot(results[i], 'ro-', label=f'{method_name} - Iter {i+1}')
    ax.set_title(f'Optimization Result: {method_name}\nDistance Function: {distance_function_name}')
    ax.set_xlabel('Index')
    ax.set_ylabel('Value')
    ax.set_xlim(0, len(P) - 1)
    ax.set_ylim(0, 6)
    ax.legend()

def create_animation(results, method_name, filename, distance_function_name):
    ani = animation.FuncAnimation(fig, animate, frames=max_iterations, fargs=(results, method_name, distance_function_name), repeat=False)
    ani.save(filename, writer='pillow', fps=4)  # 250 ms delay between frames (4 frames per second)

# Create and save the animations
create_animation(bh_results, "Basin-hopping", "basinhopping.gif", distance_function_name)
create_animation(de_results, "Differential Evolution", "differential_evolution.gif", distance_function_name)
create_animation(da_results, "Dual Annealing", "dual_annealing.gif", distance_function_name)

plt.close(fig)  # Close the plot to prevent it from displaying in the notebook


C:\Users\egorp\AppData\Local\Temp\ipykernel_60928\1725724273.py:21: RuntimeWarning: invalid value encountered in log
  return np.sum((P / (Q + epsilon)) - np.log((P + epsilon) / (Q + epsilon)) - 1)
